In [1]:
from pathlib import Path
from time import time
import pandas as pd
from videosentimentanalysis.usecases.download_video_use_case import VideoUseCase
from videosentimentanalysis.adapters.logging.custom_logger import CustomLogger
from videosentimentanalysis.adapters.logging.loguru_logger import LoguruLogger
from videosentimentanalysis.adapters.extract_audio.moviepy_extraction import MoviePyAudioExtractor
from videosentimentanalysis.usecases.video_utility_use_case import VideoUtilityUseCase
from videosentimentanalysis.adapters.transcribe_audio.speechrecognition import SpeechRecognition
from videosentimentanalysis.adapters.perform_sentiment_analysis.textblob_analysis import TextBlobAnalysis
from videosentimentanalysis.adapters.translate_text.google_translate import TextTranslation
from videosentimentanalysis.usecases.protocols.translate_text import LanguageOptions
from videosentimentanalysis.adapters.extract_emotions.text2emotion_extractor import GetEmotions

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dariavesela/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/dariavesela/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/dariavesela/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


EXPLAIN HERE THE ARCHITECTURE OF THE PROJECT

To run this project you need to have the following installed:
- ffmpeg
- python3.12
- poetry

## Setup Adapters
### Logging


In [2]:
custom_logger = CustomLogger(Path("./download_log.txt"))
loguru_logger = LoguruLogger(Path("./loguru_logger.log"))

### Video Utilities

In [3]:

moviepy_extractor = MoviePyAudioExtractor(Path("./tmp_audio"))
speechrecognition = SpeechRecognition()
sentimentanalysis = TextBlobAnalysis()
text_translation = TextTranslation()
emotion_extraction = GetEmotions()

Load all the Youtube URLs to a Pydantic Model called RawURLVideo

In [4]:
video_use_case = VideoUseCase(video_output_directory=Path("tmp_videos"), logger=custom_logger)

In [5]:
raw_url_videos=video_use_case.load_raw_url_video(Path('./video_urls.txt'))
raw_url_videos

[RawVideoUrl(url='https://www.youtube.com/watch?v=G025oxyWv0E\n'),
 RawVideoUrl(url='https://www.youtube.com/watch?v=9dgmp3KmwGg\n'),
 RawVideoUrl(url='https://www.youtube.com/watch?v=Tx1XIm6q4r4\n'),
 RawVideoUrl(url='https://www.youtube.com/watch?v=N2xk9nSEgxk\n'),
 RawVideoUrl(url='https://www.youtube.com/watch?v=NkC9AiJf7gI\n'),
 RawVideoUrl(url='https://www.youtube.com/watch?v=G7xFfezsJ1s')]

In [6]:
sequential_time_taken = time()
video_use_case.download_raw_videos_sequentially(raw_url_videos)
sequential_time_taken = time() - sequential_time_taken

2024-06-18 14:12:04.855071+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=G025oxyWv0E\n' {} ()
2024-06-18 14:12:06.834357+00:00 INFO: Downloaded video from url='https://www.youtube.com/watch?v=G025oxyWv0E\n' saved to /Users/dariavesela/VideoSentimentAnalysis/videosentimentanalysis/services/tmp_videos/c676a426-0603-4999-b64b-ff0bd8ebdde3If Educational Videos Were Filmed Like Music Videos.mp4 {} ()
2024-06-18 14:12:06.945042+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=9dgmp3KmwGg\n' {} ()
2024-06-18 14:12:08.152912+00:00 INFO: Downloaded video from url='https://www.youtube.com/watch?v=9dgmp3KmwGg\n' saved to /Users/dariavesela/VideoSentimentAnalysis/videosentimentanalysis/services/tmp_videos/c6fbeb9e-b005-4cd2-ab5c-53ccd46947f8Casually Explained Guide to College and University.mp4 {} ()
2024-06-18 14:12:08.308601+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=T

In [7]:
threading_time_taken = time()
video_use_case.download_raw_videos_threading(raw_url_videos)
threading_time_taken = time() - threading_time_taken

2024-06-18 14:12:16.895949+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=NkC9AiJf7gI\n' {} ()
2024-06-18 14:12:16.906816+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=Tx1XIm6q4r4\n' {} ()
2024-06-18 14:12:16.911928+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=N2xk9nSEgxk\n' {} ()
2024-06-18 14:12:16.921074+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=G025oxyWv0E\n' {} ()
2024-06-18 14:12:16.932179+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=9dgmp3KmwGg\n' {} ()
2024-06-18 14:12:19.095591+00:00 INFO: Downloaded video from url='https://www.youtube.com/watch?v=9dgmp3KmwGg\n' saved to /Users/dariavesela/VideoSentimentAnalysis/videosentimentanalysis/services/tmp_videos/eed796b6-2325-406b-b18a-ea7f9db4bf92Casually Explained Guide to College and University.mp4 {} ()
2024-06-18 

In [8]:
process_time_taken = time()
videos=video_use_case.download_raw_videos_multiprocessing(raw_url_videos)
process_time_taken = time() - process_time_taken

2024-06-18 14:12:22.236109+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=Tx1XIm6q4r4\n' {} ()
2024-06-18 14:12:24.918532+00:00 INFO: Downloaded video from url='https://www.youtube.com/watch?v=Tx1XIm6q4r4\n' saved to /Users/dariavesela/VideoSentimentAnalysis/videosentimentanalysis/services/tmp_videos/309a85fb-3e0e-4eef-bbfc-6125d899e700Potter Puppet Pals The Mysterious Ticking Noise.mp4 {} ()
2024-06-18 14:12:22.246837+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=G025oxyWv0E\n' {} ()
2024-06-18 14:12:24.943688+00:00 INFO: Downloaded video from url='https://www.youtube.com/watch?v=G025oxyWv0E\n' saved to /Users/dariavesela/VideoSentimentAnalysis/videosentimentanalysis/services/tmp_videos/f2bdaa69-bee4-4ec6-9da6-d97291d32a83If Educational Videos Were Filmed Like Music Videos.mp4 {} ()
2024-06-18 14:12:22.261759+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=NkC9

## Results of each method of downloading videos and time taken

In [9]:
pd.DataFrame({
    "Method": ["Sequential", "Threading", "Multiprocessing"],
    "Time Taken": [sequential_time_taken, threading_time_taken, process_time_taken]
})

,Method,Time Taken
0,Sequential,12.032584
1,Threading,5.169038
2,Multiprocessing,5.158494


Blah Blah Blah Monkey stuff

In [10]:

for video in videos:
    video_utlity_use_case=VideoUtilityUseCase(video=video, logger=loguru_logger, extract_audio=moviepy_extractor, extract_text=speechrecognition, extract_polarity_and_sensitivity=sentimentanalysis, translate=text_translation,extract_emotions=emotion_extraction)
    print(video_utlity_use_case.get_sentiment_analysis())
    print(video_utlity_use_case.extract_audio())
    print(video_utlity_use_case.transcribe_audio())
    print(video_utlity_use_case.translate_text(source_lang=LanguageOptions.ENGLISH, target_lang=LanguageOptions.SPANISH))
    print(video_utlity_use_case.extract_emotions())

2024-06-18 15:12:27.116 | INFO     | videosentimentanalysis.adapters.logging.loguru_logger:info:17 - Extracting audio from video If Educational Videos Were Filmed Like Music Videos


MoviePy - Writing audio in tmp_audio/If Educational Videos Were Filmed Like Music Videos.wav


2024-06-18 15:12:27.733 | INFO     | videosentimentanalysis.adapters.logging.loguru_logger:log:14 - Transcribing audio from video If Educational Videos Were Filmed Like Music Videos


MoviePy - Done.


2024-06-18 15:12:38.691 | INFO     | videosentimentanalysis.adapters.logging.loguru_logger:info:17 - Performing sentiment analysis on video If Educational Videos Were Filmed Like Music Videos
2024-06-18 15:12:38.710 | INFO     | videosentimentanalysis.adapters.logging.loguru_logger:info:17 - Returning cached audio for video If Educational Videos Were Filmed Like Music Videos
2024-06-18 15:12:38.711 | INFO     | videosentimentanalysis.adapters.logging.loguru_logger:info:17 - Returning cached audio for video If Educational Videos Were Filmed Like Music Videos
2024-06-18 15:12:38.711 | INFO     | videosentimentanalysis.adapters.logging.loguru_logger:log:14 - Transcribing audio from video If Educational Videos Were Filmed Like Music Videos


(0.054621848739495785, 0.054621848739495785)
local_storage_path=PosixPath('tmp_audio/If Educational Videos Were Filmed Like Music Videos.wav')


2024-06-18 15:12:48.770 | INFO     | videosentimentanalysis.adapters.logging.loguru_logger:info:17 - Returning cached audio for video If Educational Videos Were Filmed Like Music Videos
2024-06-18 15:12:48.771 | INFO     | videosentimentanalysis.adapters.logging.loguru_logger:log:14 - Transcribing audio from video If Educational Videos Were Filmed Like Music Videos


 The opening shot is a little confusing and a little unreal. That sense of unreality is because nearly every shot in this video is being played back at the wrong speed. We filmed this section at 30 frames a second, but it's being played back at 24, 80% of normal speed. So it just seems a little bit off. Plus we're using rapid cuts with no continuity between them, different clothes, different locations, different everything. We've also got some cutaways in extreme slow motion, because it looks spectacular, and because it lets me do a moody stare into the distance without having to hold it for so long that it becomes awkward. It also lets us fit in some product placement. Some of the shots aren't slow motion though. This is being played back faster than real time, so that the dancers seem more synchronised and precise than they really are. I mean, don't get me wrong, they're good, but would the speed change their movements look superhuman? Also, there are dancers, because having attracti

TypeError: 'GetEmotions' object is not callable